In [32]:
import numpy as np
from dipy.io.streamline import load_trk
from dipy.viz import window, actor
from dipy.tracking.streamline import Streamlines
import math

In [2]:
def show_streamlines(streamlines): #function to visualize streamlines
    
    ren = window.Renderer()
    ren.add(actor.line(streamlines))
    window.show(ren)

In [3]:
def show_multiple_streamlines(streamline1,streamline2): #function to visualize streamlines
    
    ren = window.Renderer()
    ren.add(actor.line(streamline1))
    ren.add(actor.line(streamline2,colors=(0,1,1)))
    window.show(ren)

 load brain (tractogram)

In [4]:
brain, b_hr = load_trk("brain.trk")

load Arcuate Fasciculus left fiber bundle (AF_L)

In [5]:
bundle, bn_hr = load_trk("AF_L.trk")

In [6]:
#cst, bn_hr = load_trk("CST_recognized.trk")

let's visualize them

In [7]:
# first brain

#show_streamlines(brain)

In [8]:
# now bundle

show_streamlines(bundle[:50])

/Users/rp7/anaconda3/lib/python3.6/site-packages/vtk/util/numpy_support.py:137: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  assert not numpy.issubdtype(z.dtype, complex), \


In [9]:
(bundle[0].shape)

(331, 3)

In [10]:
from sklearn.decomposition import PCA
def project2D(streamline):
    pca = PCA(n_components=2)
    newBundle = pca.fit_transform(streamline)
    newStreamline = Streamlines([np.array((np.hstack((newBundle,np.zeros((newBundle.shape[0])).reshape((-1,1))))), dtype=float)])
    return newStreamline, pca.explained_variance_ratio_, pca.singular_values_

In [35]:
# print(pca.explained_variance_ratio_) 
# print(pca.singular_values_)

In [14]:
# st = Streamlines([newBundle])
for streamline in bundle:
    newStreamline, varRatio, singVal = project2D(streamline)
    # append these values to store them 
show_multiple_streamlines(newStreamline,bundle[0:1])

/Users/rp7/anaconda3/lib/python3.6/site-packages/vtk/util/numpy_support.py:137: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  assert not numpy.issubdtype(z.dtype, complex), \


In [33]:
angles = np.zeros(newBundle.shape[0])
angles[0] = 0
for point in range(1,newBundle.shape[0]):
    diff = newBundle[point] - newBundle[point-1]
    angles[point] = math.degrees(math.atan(diff[1]/diff[0]))

In [34]:
angles

array([  0.        ,  53.32359583,  53.33113623,  53.32889023,
        53.32808629,  53.3287197 ,  53.32935065,  53.32900716,
        53.3287197 ,  53.32946759,  53.32825926,  53.32889023,
        53.04552909,  53.04478604,  53.04524273,  53.04561549,
        53.04495885,  53.04524273,  53.04532913,  64.426705  ,
        64.42604318,  64.42784021,  64.42569953,  58.38867572,
        58.38733584,  58.38772993,  58.38836046,  58.38757229,
        58.38749348,  58.38910919,  58.38757229,  58.82709988,
        58.82610981,  58.10189322,  58.10328154,  58.10328154,
        58.10245771,  58.10320526,  58.10279335,  58.10245771,
        58.1032434 ,  58.10275521,  58.10343409,  52.15767474,
        52.15802692,  52.15884437,  52.15798322,  52.15830455,
        52.15839195,  52.15793952,  52.15752821,  52.15884437,
        52.15789325,  52.15789325,  48.33740043,  48.33680887,
        48.33633199,  48.33740043,  48.33680887,  50.99874707,
        50.99861181,  50.99781917,  50.99856583,  50.99

In [16]:
# show_streamlines(np.hstack((newBundle,np.zeros((newBundle.shape[0])).reshape((-1,1)))))

In [17]:
#show_streamlines(cst)

In [18]:
#show_multiple_streamlines(bundle,cst)

let's see its data type

In [19]:
#bundle # whole bundle, made of several streamlines

In [20]:
# ((brain)[0]).shape# number of streamlines in bundle

In [21]:
# bundle[1] #one streamline in bundle
# (np.array(bundle)[0]).shape
# np.array(bundle)

In [22]:
# (bundle[1].shape) #one point on one streamline

In [23]:
npbrain = np.array(brain)

In [24]:
# npbrain[0].shape

In [25]:
# len(bundle)

these are x, y and z co-ordinates of the point

In [18]:
# set labels
labels = np.zeros(len(brain))
for i in range(len(bundle)):
    for j in range(len(brain)):
        if((npbrain[j].shape[0]==bundle[i].shape[0])):
#         print(i,j)
            if np.allclose(npbrain[j], bundle[i]):
                labels[j] = 1
                break
    if(i%100==0):
        print(i,end=" ")
                

0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 

In [27]:
# npbrain[0].shape

In [24]:
n_pts = [len(streamline) for streamline in bundle]

In [25]:
from dipy.tracking.streamline import (set_number_of_points, nbytes,select_random_set_of_streamlines)
newStreamlines = set_number_of_points(bundle,101)
n_pts_ds = [len(streamline) for streamline in newStreamlines]

In [26]:
newBrain = set_number_of_points(brain,101)

In [28]:
# newBrain[0].shape

In [29]:
# len(npbrain)

In [29]:
import tensorflow as tf
import keras

Using TensorFlow backend.


In [60]:
# ==========
#   MODEL
# ==========

# Parameters
learning_rate = 0.02
training_steps = 70
batch_size = 100
display_step = 200

# Network Parameters
seq_max_len = 1000 # Sequence max length
n_hidden = 64 # hidden layer num of features
n_classes = 2 # linear sequence or not

trainset = npbrain[1950:2050]
testset = labels[1950:2050]

# tf Graph input
x = tf.placeholder("float", [None, seq_max_len, 3])
y = tf.placeholder("float", [None, n_classes])
# A placeholder for indicating each sequence length
seqlen = tf.placeholder(tf.int32, [None])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [31]:
def dynamicRNN(x, seqlen, weights, biases):
    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)
    
    # Unstack to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, seq_max_len, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden)

    # Get lstm cell output, providing 'sequence_length' will perform dynamic
    # calculation.
    outputs, states = tf.contrib.rnn.static_rnn(lstm_cell, x, dtype=tf.float32,
                                sequence_length=seqlen)

    # When performing dynamic calculation, we must retrieve the last
    # dynamically computed output, i.e., if a sequence length is 10, we need
    # to retrieve the 10th output.
    # However TensorFlow doesn't support advanced indexing yet, so we build
    # a custom op that for each sample in batch size, get its length and
    # get the corresponding relevant output.

    # 'outputs' is a list of output at every timestep, we pack them in a Tensor
    # and change back dimension to [batch_size, n_step, n_input]
    outputs = tf.stack(outputs)
    outputs = tf.transpose(outputs, [1, 0, 2])

    # Hack to build the indexing and retrieve the right output.
    batch_size = tf.shape(outputs)[0]
    # Start indices for each sample
    index = tf.range(0, batch_size) * seq_max_len + (seqlen - 1)
    # Indexing
    outputs = tf.gather(tf.reshape(outputs, [-1, n_hidden]), index)

    # Linear activation, using outputs computed above
    return tf.matmul(outputs, weights['out']) + biases['out']

In [32]:
tf_session = keras.backend.get_session()
tf_graph = tf.get_default_graph()
# tf.reset_default_graph()
print([n.name for n in tf.get_default_graph().as_graph_def().node])
# tf.contrib.rnn.reset_states()
# with tf_session.as_default():
#      with tf_graph.as_default():
#             tf.reset_default_graph() 
pred = dynamicRNN(x, seqlen, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

['Placeholder', 'Placeholder_1', 'Placeholder_2', 'random_normal/shape', 'random_normal/mean', 'random_normal/stddev', 'random_normal/RandomStandardNormal', 'random_normal/mul', 'random_normal', 'Variable', 'Variable/Assign', 'Variable/read', 'random_normal_1/shape', 'random_normal_1/mean', 'random_normal_1/stddev', 'random_normal_1/RandomStandardNormal', 'random_normal_1/mul', 'random_normal_1', 'Variable_1', 'Variable_1/Assign', 'Variable_1/read', 'IsVariableInitialized', 'IsVariableInitialized_1', 'init']
Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



/Users/rp7/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [62]:
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, training_steps+1):
#         batch_x, batch_y, batch_seqlen = trainset.next(batch_size)
        rangelow = ((step-1)*batch_size)%(trainset.shape[0])
        rangehigh = ((step)*batch_size)%(trainset.shape[0])
        if(rangehigh==rangelow):
            rangehigh=rangehigh-1
        if(rangehigh<rangelow):
            batch_x = np.append(trainset[rangelow:], trainset[:rangehigh])
            batch_y = np.append(testset[rangelow:], testset[:rangehigh])
        else:
            batch_x = trainset[rangelow:rangehigh]
            batch_y = testset[rangelow:rangehigh]
        
        batch_seqlen = []
        for sample in trainset:
            batch_seqlen.append(sample.shape[0])

#         batch_seqlen = batch_seqlen.tolist()
        print(type(batch_seqlen))
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y,
                                       seqlen: batch_seqlen})
        if step % display_step == 0 or step == 1:
            # Calculate batch accuracy & loss
            acc, loss = sess.run([accuracy, cost], feed_dict={x: batch_x, y: batch_y,
                                                seqlen: batch_seqlen})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy
    test_data = testset.data
    test_label = testset.labels
    test_seqlen = testset.seqlen
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_data, y: test_label,
                                      seqlen: test_seqlen}))

<class 'list'>


AttributeError: module 'tensorflow' has no attribute 'identify'

In [37]:
class ToySequenceData(object):
    """ Generate sequence of data with dynamic length.
    This class generate samples for training:
    - Class 0: linear sequences (i.e. [0, 1, 2, 3,...])
    - Class 1: random sequences (i.e. [1, 3, 10, 7,...])

    NOTICE:
    We have to pad each sequence to reach 'max_seq_len' for TensorFlow
    consistency (we cannot feed a numpy array with inconsistent
    dimensions). The dynamic calculation will then be perform thanks to
    'seqlen' attribute that records every actual sequence length.
    """
    def __init__(self, n_samples=1000, max_seq_len=20, min_seq_len=3,
                 max_value=1000):
        self.data = []
        self.labels = []
        self.seqlen = []
        for i in range(n_samples):
            # Random sequence length
            len = np.random.randint(min_seq_len, max_seq_len)
            # Monitor sequence length for TensorFlow dynamic calculation
            self.seqlen.append(len)
            # Add a random or linear int sequence (50% prob)
            if np.random.random() < .5:
                # Generate a linear sequence
                rand_start = np.random.randint(0, max_value - len)
                s = [[float(i)/max_value] for i in
                     range(rand_start, rand_start + len)]
                # Pad sequence for dimension consistency
                s += [[0.] for i in range(max_seq_len - len)]
                self.data.append(s)
                self.labels.append([1., 0.])
            else:
                # Generate a random sequence
                s = [[float(np.random.randint(0, max_value))/max_value]
                     for i in range(len)]
                # Pad sequence for dimension consistency
                s += [[0.] for i in range(max_seq_len - len)]
                self.data.append(s)
                self.labels.append([0., 1.])
        self.batch_id = 0

    def next(self, batch_size):
        """ Return a batch of data. When dataset end is reached, start over.
        """
        if self.batch_id == len(self.data):
            self.batch_id = 0
        batch_data = (self.data[self.batch_id:min(self.batch_id +
                                                  batch_size, len(self.data))])
        batch_labels = (self.labels[self.batch_id:min(self.batch_id +
                                                  batch_size, len(self.data))])
        batch_seqlen = (self.seqlen[self.batch_id:min(self.batch_id +
                                                  batch_size, len(self.data))])
        print(type(batch_seqlen[0]))
        self.batch_id = min(self.batch_id + batch_size, len(self.data))
        return batch_data, batch_labels, batch_seqlen

In [38]:
tr = ToySequenceData(n_samples=1000, max_seq_len=seq_max_len)
te = ToySequenceData(n_samples=500, max_seq_len=seq_max_len)

In [39]:
b_x, b_y, b_seqlen = tr.next(100)

<class 'int'>


In [47]:
print((b_x))
# print(batch_seqlen)

[[[0.213], [0.214], [0.215], [0.216], [0.217], [0.218], [0.219], [0.22], [0.221], [0.222], [0.223], [0.224], [0.225], [0.226], [0.227], [0.228], [0.229], [0.23], [0.231], [0.232], [0.233], [0.234], [0.235], [0.236], [0.237], [0.238], [0.239], [0.24], [0.241], [0.242], [0.243], [0.244], [0.245], [0.246], [0.247], [0.248], [0.249], [0.25], [0.251], [0.252], [0.253], [0.254], [0.255], [0.256], [0.257], [0.258], [0.259], [0.26], [0.261], [0.262], [0.263], [0.264], [0.265], [0.266], [0.267], [0.268], [0.269], [0.27], [0.271], [0.272], [0.273], [0.274], [0.275], [0.276], [0.277], [0.278], [0.279], [0.28], [0.281], [0.282], [0.283], [0.284], [0.285], [0.286], [0.287], [0.288], [0.289], [0.29], [0.291], [0.292], [0.293], [0.294], [0.295], [0.296], [0.297], [0.298], [0.299], [0.3], [0.301], [0.302], [0.303], [0.304], [0.305], [0.306], [0.307], [0.308], [0.309], [0.31], [0.311], [0.312], [0.313], [0.314], [0.315], [0.316], [0.317], [0.318], [0.319], [0.32], [0.321], [0.322], [0.323], [0.324], [0